In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [37]:
df = pd.read_csv('C:\\Users\\M\\Machine_L\\data\\titanic_train.csv', index_col = 0)
df.head(2)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


## svc 알고리즘으로 titanic 생존자 예측하는 모델을 생성하고 파라미터 최적화를 통해서 모델의 성능을 최적화하시오.

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [39]:
#1. 전처리
df.drop(columns = ['Cabin'], inplace = True)
df.dropna(inplace = True)
df_ohe = pd.get_dummies(df, dtype = 'int')

In [ ]:
# Age nan값 채우기
df.Age.fillna(df.groupby('Pclass').Age.transform('mean'))

In [ ]:
df.isnull().sum()

In [41]:
#2. X, y 설정
X = df_ohe.drop(columns = ['Survived'])
y = df_ohe.Survived

In [42]:
#3. train, test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 9999)

In [43]:
#4. 모델 학습
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train)

SVC()

In [44]:
#5. 예측
pred = svc.predict(X_test)

In [45]:
#6. 검증
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.68      0.86      0.76        92
           1       0.50      0.25      0.34        51

    accuracy                           0.64       143
   macro avg       0.59      0.56      0.55       143
weighted avg       0.61      0.64      0.61       143



In [33]:
from sklearn.metrics import accuracy_score
ac = accuracy_score(y_test, pred)
print(ac)

0.6123595505617978


In [34]:
# 7. 파라미터 최적화
from sklearn.model_selection import GridSearchCV

params = {'C':[0.001, 0.01, 0.1, 1, 10, 100], 'gamma':[0.00001, 0.0001, 0.01, 0.1, 1, 10, 100]} 
grid = GridSearchCV(SVC(), param_grid = params, refit = True, verbose = True)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'gamma': [1e-05, 0.0001, 0.01, 0.1, 1, 10, 100]},
             verbose=True)

In [35]:
bp = grid.best_params_
bs = grid.best_score_
print('best param : ', bp)
print('best score : ', bs)

best param :  {'C': 100, 'gamma': 0.01}
best score :  0.7988870284644933
